# Preparação de dados e Modelagem
A partir dos dados meteorológicos, prever a quantidade de pessoas que buscarão abrigo

0. [Preparação do df](#id0) <br>
1. [Divisão entre treino e teste](#id1) <br>
2. [Tratamento de outliers](#id2) <br>
3. [Separação da target das demais variáveis](#id3) <br>
4. [Balanceamento 1:1](#id4) <br>

In [23]:
# Importando bibliotecas
import sweetviz as sv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels
from statsmodels.stats.outliers_influence import variance_inflation_factor
from unidecode import unidecode
import random

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', None)

var_target = 'qtd_pessoas'

#### Lendo CSVs obtido ao final do notebook de análise exploratória e engenharia de variáveis

In [24]:
df = pd.read_csv('df.csv')

In [25]:
df.columns

Index(['id', 'situac', 'onde_dormiu_ontem', 'tempo_perman', 'sexo',
       'sexo_nasc', 'idade', 'faixa_idade', 'class_idade', 'cor_raca_etnia',
       'parentes_ca', 'tempo_situac', 'tipo_ponto', 'moradia_impro',
       'crianca_acomp', 'crianca_sozinha', 'lat', 'lon', 'Data', 'subpref',
       'distrito', 'precipitacao_total', 'temp_max', 'temp_min', 'umidade_max',
       'umidade_min', 'vel_max_vento'],
      dtype='object')

In [26]:
df.drop(columns=['situac', 'onde_dormiu_ontem', 'tempo_perman', 'sexo', 'sexo_nasc', 'idade', 'faixa_idade', 'class_idade', 'cor_raca_etnia',
                 'parentes_ca', 'tempo_situac', 'tipo_ponto', 'moradia_impro', 'crianca_acomp', 'crianca_sozinha', 'lat', 'lon', 'subpref',
                 'distrito'], inplace = True) 

<a id="id0"></a>

### Preparar o df

In [5]:
# Agrupar o DataFrame por data
df = df.groupby("Data").agg({"id": "count",
                              "precipitacao_total": "max",
                              "temp_max": "max",
                              "temp_min": "min",
                              "umidade_max": "max",
                              "umidade_min": "min",
                              "umidade_min": "max",
                             })

# Redefina o índice do DataFrame agrupado
df = df.reset_index()
df = df.rename(columns={"id": "qtd_pessoas"})

df

,Data,qtd_pessoas,precipitacao_total,temp_max,temp_min,umidade_max,umidade_min
0,03/11/2021,5249,1.00,22.20,16.80,100.00,69.00
1,04/11/2021,4660,0.00,26.10,16.20,100.00,39.00
2,05/11/2021,1724,0.00,23.50,16.20,100.00,62.00
3,08/11/2021,4204,0.00,22.70,14.90,100.00,58.00
4,09/11/2021,2099,0.00,19.40,16.30,100.00,85.00
5,10/11/2021,1865,0.00,21.20,15.90,100.00,67.00
6,11/10/2021,3,10.60,18.70,15.50,100.00,85.00
7,11/11/2021,855,2.80,17.00,14.90,100.00,91.00
8,13/11/2021,4,0.00,18.20,12.50,93.00,50.00
9,14/10/2021,138,0.60,30.20,16.30,100.00,38.00


In [6]:
df.columns

Index(['Data', 'qtd_pessoas', 'precipitacao_total', 'temp_max', 'temp_min',
       'umidade_max', 'umidade_min'],
      dtype='object')

Temos apenas 20 dias, isso sem retirar aqueles dias com poucos cadastros, mas iremos nos limitar a base que temos acesso

### Colocando a temperatura do dia anterior

In [7]:
df['precipitacao_total'] = df['precipitacao_total'].shift(1)
df['temp_max'] = df['temp_max'].shift(1)
df['temp_min'] = df['temp_min'].shift(1)
df['umidade_max'] = df['umidade_max'].shift(1)
df['umidade_min'] = df['umidade_min'].shift(1)

In [9]:
df = df.drop(index=0).reset_index(drop=True)
df

,Data,qtd_pessoas,precipitacao_total,temp_max,temp_min,umidade_max,umidade_min
0,04/11/2021,4660,1.00,22.20,16.80,100.00,69.00
1,05/11/2021,1724,0.00,26.10,16.20,100.00,39.00
2,08/11/2021,4204,0.00,23.50,16.20,100.00,62.00
3,09/11/2021,2099,0.00,22.70,14.90,100.00,58.00
4,10/11/2021,1865,0.00,19.40,16.30,100.00,85.00
5,11/10/2021,3,0.00,21.20,15.90,100.00,67.00
6,11/11/2021,855,10.60,18.70,15.50,100.00,85.00
7,13/11/2021,4,2.80,17.00,14.90,100.00,91.00
8,14/10/2021,138,0.00,18.20,12.50,93.00,50.00
9,15/10/2021,292,0.60,30.20,16.30,100.00,38.00


<a id="id1"></a>

### Divide em treino e teste
A divisão atual é:
- Treino: 70% da base
- Teste: 30% da base

In [10]:
df_train, df_test = train_test_split(df, test_size=0.30, random_state=42)

<a id="id2"></a>

### Separação da variável alvo das demais variáveis de treino
Essa separação é necessária por se tratar de um input obrigatório do modelo.
- X -> apresenta todas as variáveis explicativas selecionadas
- y -> apresenta apenas a variável target

In [11]:
#Treino
#Com outlier
X_train = df_train.copy()
X_train.drop(columns=[var_target], inplace = True) 

y_train = df_train[[var_target]]


#Teste
X_test = df_test.copy()
X_test.drop(columns=[var_target], inplace = True) 

y_test = df_test[[var_target]]

del df_train, df_test

In [12]:
y_train.head()

,qtd_pessoas
3,2099
13,716
15,1281
17,676
2,4204


In [13]:
y_test.head()

,qtd_pessoas
0,4660
5,3
11,453
1,1724
8,138


#### Salva os CSVs para serem utilizados nos notebooks seguintes

In [14]:
#Treino
X_train.to_csv('X_train.csv', index=False)
y_train.to_csv('y_train.csv', index=False)

#Teste
X_test.to_csv('X_test.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

#### 1.2 Salvando a coluna CD_CLI
Este processo é feito para identificar a data em questão

In [15]:
#Treino
col_data_train = X_train[['Data']]
X_train.drop(columns='Data', inplace=True)

#Teste
col_data_test = X_test[['Data']]
X_test.drop(columns='Data', inplace=True)

In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Crie um objeto de regressão linear
reg = LinearRegression()

# Treine o modelo usando os dados de treino
reg.fit(X_train, y_train)

# Faça previsões com o modelo treinado no conjunto de teste
y_pred = reg.predict(X_test)

# Calcule o coeficiente de determinação (R²) do modelo no conjunto de teste
r2 = r2_score(y_test, y_pred)
print("Coeficiente de Determinação (R²): {:.2f}".format(r2))

# Calcule o erro médio absoluto do modelo no conjunto de teste
mae = mean_absolute_error(y_test, y_pred)
print("Erro Médio Absoluto (MAE): {:.2f}".format(mae))

# Calcule o erro médio quadrático do modelo no conjunto de teste
mse = mean_squared_error(y_test, y_pred)
print("Erro Médio Quadrático (MSE): {:.2f}".format(mse))

Coeficiente de Determinação (R²): -0.30
Erro Médio Absoluto (MAE): 1582.88
Erro Médio Quadrático (MSE): 3307458.59


In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Defina o modelo de regressão linear
reg = LinearRegression()

# Defina os hiperparâmetros que deseja ajustar
param_grid = {
    'normalize': [True, False],
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [-1, None],
    'positive': [True, False]
}

# Defina as métricas de avaliação que deseja utilizar
scoring = {
    'r2': 'r2',
    'mae': 'neg_mean_absolute_error',
    'mse': 'neg_mean_squared_error'
}

# Execute o GridSearchCV para encontrar a melhor combinação de hiperparâmetros
grid_search = GridSearchCV(reg, param_grid, cv=5, scoring=scoring, refit='r2')
grid_search.fit(X_train, y_train)

# Imprima os resultados
print("Melhores hiperparâmetros: ", grid_search.best_params_)
print("R²: {:.2f}".format(grid_search.best_score_))
print("MAE: {:.2f}".format(-grid_search.cv_results_['mean_test_mae'][grid_search.best_index_]))
print("MSE: {:.2f}".format(-grid_search.cv_results_['mean_test_mse'][grid_search.best_index_]))


C:\Users\JG956GV\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\JG956GV\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklear

Melhores hiperparâmetros:  {'copy_X': True, 'fit_intercept': False, 'n_jobs': -1, 'normalize': True, 'positive': False}
R²: -936.81
MAE: 1647.39
MSE: 4494603.39


C:\Users\JG956GV\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\JG956GV\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklear

In [18]:
# Importar bibliotecas necessárias
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Criar um modelo de Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Treinar o modelo de Random Forest
rf_model.fit(X_train, y_train)

# Fazer a previsão usando seus dados de teste
y_pred = rf_model.predict(X_test)

# Avaliar o desempenho do modelo usando métricas como R², MAE e MSE
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Coeficiente de Determinação (R²): {:.2f}".format(r2))
print("Erro Médio Absoluto (MAE): {:.2f}".format(mae))
print("Erro Médio Quadrático (MSE): {:.2f}".format(mse))


Coeficiente de Determinação (R²): -0.78
Erro Médio Absoluto (MAE): 1586.11
Erro Médio Quadrático (MSE): 4517379.43


C:\Users\JG956GV\AppData\Local\Temp\ipykernel_77220\3501232393.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


In [19]:
# Importar bibliotecas necessárias
import lightgbm as lgb
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


# Criar objeto Dataset do LightGBM usando seus dados de treinamento
lgb_train = lgb.Dataset(X_train, y_train)

# Definir os parâmetros do modelo
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# Treinar o modelo usando o objeto Dataset
lgb_model = lgb.train(params, lgb_train, num_boost_round=100)

# Fazer a previsão usando seus dados de teste
y_pred = lgb_model.predict(X_test)

# Avaliar o desempenho do modelo usando métricas como R², MAE e MSE
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Coeficiente de Determinação (R²): {:.2f}".format(r2))
print("Erro Médio Absoluto (MAE): {:.2f}".format(mae))
print("Erro Médio Quadrático (MSE): {:.2f}".format(mse))


[LightGBM] [Warning] There are no meaningful features, as all feature values are constant.
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that m

In [21]:
# Importe as bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Crie uma instância do modelo XGBRegressor
xgb = XGBRegressor()

# Defina uma lista de parâmetros para tunar
params = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [50, 100, 200],
    'gamma': [0, 0.1, 0.5],
    'subsample': [0.5, 0.7, 1],
    'colsample_bytree': [0.5, 0.7, 1]
}

# Use GridSearchCV para encontrar os melhores parâmetros
grid_search = GridSearchCV(xgb, param_grid=params, cv=5)
grid_search.fit(X_train, y_train)

# Use o melhor modelo encontrado pelo GridSearchCV
xgb_best = grid_search.best_estimator_

# Treine o modelo no conjunto de treino com os melhores parâmetros
xgb_best.fit(X_train, y_train)

# Faça previsões com o modelo treinado no conjunto de teste
y_pred = xgb_best.predict(X_test)

# Calcule o coeficiente de determinação (R²) do modelo no conjunto de teste
r2 = r2_score(y_test, y_pred)
print("Coeficiente de Determinação (R²): {:.2f}".format(r2))


Coeficiente de Determinação (R²): -0.86


In [22]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

# Normalizar os dados meteorológicos
scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

# Criar um modelo KNN
knn = KNeighborsRegressor()

# Definir os parâmetros a serem testados
params = {
    'n_neighbors': [5, 10, 20],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

# Ajustar o modelo usando GridSearchCV
grid_search = GridSearchCV(knn, params, cv=5)
grid_search.fit(X_train_norm, y_train)

# Fazer previsões no conjunto de teste
y_pred = grid_search.predict(X_test_norm)

# Calcular o coeficiente de determinação (R²) do modelo no conjunto de teste
r2 = r2_score(y_test, y_pred)
print("Coeficiente de Determinação (R²): {:.2f}".format(r2))


Coeficiente de Determinação (R²): 0.00


C:\Users\JG956GV\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\JG956GV\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\JG956GV\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "C:\Users\JG956GV\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py", line 720, in score
    y_pred = self.predict(X)
  File "C:\Users\JG956GV\AppData\Roaming\Python\Python39\site-packages\sklearn\neighbors\_regression.py", line 229, in predict
    neigh_ind = self.kneighbors(X, return_distance=False)
  File "C:\Users\JG956GV\AppData\Roaming\Python\Python39\site